In [1]:
import robot
import map
import utils

import numpy as np
import gtsam
import matplotlib.pyplot as plt

%matplotlib qt

# unitTest 1: Move robot and check odometrey measurements

In [4]:
car = robot.robot()

#control inputs
dr = 0.4 #[m]
dtheta = np.radians(20)

time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()

#set graphics
fig , ax = utils.setWorldMap()
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)
ax.legend(["ground truth","dead reckoning"])

#run and plot simulation
xcurrent_DR = hist_GT.copy()
while time <= timeFinal:
    meas_odom = car.moveAndMeasureOdometrey((dr,dtheta))
    
    #dead reckoning integration
    xcurrent_DR[2] += meas_odom.dtheta #update angle
    xcurrent_DR[0] += meas_odom.dr * np.cos(xcurrent_DR[2])
    xcurrent_DR[1] += meas_odom.dr * np.sin(xcurrent_DR[2])
    
    #log history
    hist_DR = np.vstack([hist_DR,xcurrent_DR])
    hist_GT = np.vstack([hist_GT,car.pose])

    #increase time
    time += dt

    #plot
    graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
    graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
    plt.draw()
    plt.pause(dt)

# unitTest 2: Build prior map

In [5]:
np.random.seed(seed=2)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigMin = -0.5
sigMax = 0.5

landmarks = [None] * N
for ii in range(N):
    sigs = np.random.uniform(low=sigMin, high=sigMax, size=(2,2))
    cov = sigs@sigs.T #enforce symmetric and positive definite: https://mathworld.wolfram.com/PositiveDefiniteMatrix.html
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 cov = cov)

priorMap = map.map(landmarks)
priorMap.plot(plotIndex = True,plotCov = True)

# unitTest 3: Move car and check landmark measurements

In [ ]:
#------Build worldmap
fig , ax = utils.setWorldMap()
np.random.seed(seed=0)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)

landmarks = [None] * N
for ii in range(N):
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 )
worldMap = map.map(landmarks)
worldMap.plot(ax = ax, plotIndex = True,plotCov = False)

#------Spawn Robot
car = robot.robot()
#control inputs
dr = 0.4 #[m]
dtheta = np.radians(20)
odom = (dr,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)

#run and plot simulation
xcurrent_DR = hist_GT.copy()
while time <= timeFinal:
    meas_odom = car.moveAndMeasureOdometrey((dr,dtheta))

    meas_lm = car.measureLandmarks(worldMap)

    #dead reckoning integration
    xcurrent_DR[2] += meas_odom.dtheta #update angle
    xcurrent_DR[0] += meas_odom.dr * np.cos(xcurrent_DR[2])
    xcurrent_DR[1] += meas_odom.dr * np.sin(xcurrent_DR[2])
    
    #log history
    hist_GT = np.vstack([hist_GT,car.pose])
    hist_DR = np.vstack([hist_DR,xcurrent_DR])
    hist_LM.append(meas_lm)

    #increase time
    time += dt

    #plot
    graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
    graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
    plt.draw()
    plt.pause(dt)

print(hist_LM)

# unitTest4: Plot Robot Cone

In [ ]:
#------Build worldmap
fig , ax = utils.setWorldMap()
np.random.seed(seed=0)

#------Spawn Robot
car = robot.robot()
car.plot(ax)
#control inputs
dr = 0.4 #[m]
dtheta = np.radians(20)
odom = (dr,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

while time <= timeFinal:
    car.moveAndMeasureOdometrey((dr,dtheta))

    #increase time
    time += dt

    car.plot()
    plt.draw()
    plt.pause(dt)

# unitTest5: putting everything together before gtsam

In [3]:
car, worldMap, ax, fig = utils.default_world()
prms = utils.default_parameters()

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 1)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 1)

#run and plot simulation
time = 0
xcurrent_DR = hist_GT.copy()
while time <= prms["time"]["timeFinal"]:
    meas_odom = car.moveAndMeasureOdometrey((prms["odom"]["dr"],prms["odom"]["dtheta"]))

    meas_lm = car.measureLandmarks(worldMap)

    #dead reckoning integration
    xcurrent_DR[2] += meas_odom.dtheta #update angle
    xcurrent_DR[0] += meas_odom.dr * np.cos(xcurrent_DR[2])
    xcurrent_DR[1] += meas_odom.dr * np.sin(xcurrent_DR[2])
    
    #log history
    hist_GT = np.vstack([hist_GT,car.pose])
    hist_DR = np.vstack([hist_DR,xcurrent_DR])
    hist_LM.append(meas_lm)

    #increase time
    time += prms["time"]["dt"]

    #plot
    car.plot()
    graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
    graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
    plt.draw()
    plt.pause(prms["time"]["dt"])

print(hist_LM[0])

[<robot.meas_landmark object at 0x7f5da4e00430>, <robot.meas_landmark object at 0x7f5da4e00310>, <robot.meas_landmark object at 0x7f5da4e002e0>, <robot.meas_landmark object at 0x7f5da4e00400>, <robot.meas_landmark object at 0x7f5dac037340>, <robot.meas_landmark object at 0x7f5dac037c40>, <robot.meas_landmark object at 0x7f5dac0372e0>, <robot.meas_landmark object at 0x7f5da4e27040>]
